## Curso Python para Economistas
### Trabajo Práctico Nº 6 (integrador)

### Fecha de entrega:
Martes 12/11 a las 23:59 hs\*

\* *tienen una semana más que lo usual para resolver este TP*

### Modalidad de entrega y trabajo
- Este TP es **individual**.
- Un repositorio **privado** debe ser creado en GitHub para el TP, y dar acceso a los 5 profesores. Los nombres de usuarios son: `belenmichel`, `rbonazzola`, `Queeno11`, `agoszulli` y `pilarch`.
- Cuando hayan hecho ese último commit, copien la URL para clonar su repositorio y péguenla en
[este Google Sheet](https://docs.google.com/spreadsheets/d/1byFOKyV7UnBuPuF9X2JWFVOUZX36b2_CEDw9ColP7VQ/edit?usp=sharing), en la hoja del TP6. Al ser un repositorio privado, solo los colaboradores habilitados podrán clonarlo.
- Al finalizar el trabajo práctico deben hacer un último commit y push en su repositorio de GitHub con el mensaje `"Entrega final del TP6"`. Antes de la fecha y hora de entrega pueden hacer cuantos cambios quieran en el repositorio, pero luego de la hora de corte no deben hacer más cambios. Si un commit con el mensaje anterior se realiza luego de la hora de entrega, se supondrá que la entrega tardía fue intencional y se utilizarán los días de gracia. La última versión en el repositorio es la que será evaluada. Para esto es importante que no completen el Google Sheet hasta que no hayan finalizado el TP, como tampoco hacer pushes posteriores a la entrega.
- Comentar su código y utilizar buenas prácticas en el estilo del mismo tendrá un peso en la nota de este trabajo (1pto por inciso). Pueden utilizar esta [guía de estilo](https://recursospython.com/pep8es.pdf) como referencia.

## Consignas

#### Parte 1:
Esta parte del trabajo es abierta: los detalles de las consignas no están definidas al 100% y deberán usar sus intereses y criterios para guiarlos.
El objetivo de esta parte es que pongan a prueba las habilidades desarrolladas para conectarse a una API REST que sea de su interés. Nuestra sugerencia es que comiencen a pensar en datos con los cuales les gustaría trabajar en sus tesis y que exploren nuevas páginas web y datos con los que no han trabajado anteriormente. 

1. **Elección de una API REST** (5 puntos): Elijan una API pública que sea de interés para ustedees y justifiquen por qué les resulta interesante trabajar con los datos de esa API. Detallen la pregunta de investigación que les gustaría responder con esos datos. 

   Aquí tienen algunas sugerencias:

- [API de datos.gob.ar](https://datos.gob.ar/apis) (datos de Argentina)
- [IDB API](https://mydata.iadb.org/) (indicadores económicos y + de America Latina y el Caribe)
- [World Bank API](https://data.worldbank.org/indicator) (indicadores económicos globales)
- O pueden buscar otras APIs en [este link](https://github.com/public-apis/)

Respuesta: uso la API de Series de Tiempo AR, ayudándome del generador de URLs (https://datosgobar.github.io/series-tiempo-ar-call-generator/) que me permite ver todas las series disponibles e identificar muy fácilmente la URL deseada.
Elijo la serie del Índice del Costo de la Construcción por gran componente, que en realidad está compuesto por 3 series de tiempo: una para el componente materiales, otra para la mano de obra y otra que indica el nivel general. 
Me interesa ver cómo fue la variación temporal de estas tres, especialmente ver la diferencia en la variación de cada uno de los componentes, ya que esto nos puede dar indicios de la evolución de los salarios de los empleados de la construcción, así como el peso relativo de estos en los costos totales del rubro.

2. **Conectar con la API y extraer datos** (5 puntos)
    - Hagan una solicitud a la API utilizando el módulo `requests` de Python.
    - Definan parámetros adecuados para obtener un conjunto de datos relevante para el uso que has planteado.
    - Asegúrense de manejar correctamente el caso en que la solicitud falle (por ejemplo, con un mensaje de error o un valor predeterminado).

In [1]:
import requests
import pandas as pd

In [2]:
def get_costo_construccion(componente):
    # Me creo un diccionario con los valores que puede tomar componente como key y el texto de la url correspondiente como value
    componentes = {
        "Mano de obra": "aRxVTe",
        "Materiales": "PvexBm",
        "Nivel general": "HydoB9"
    }

    # Como la forma de escribir el componente puede variar, me aseguro de que sea accesible incluso cuando no me acuerdo 
    # exactamente como escribirlos o cuáles son los valores posibles
    if componente in componentes:
        #extraigo el texto a insertar en la url correspondiente al valor de componente que pasé como parámetro al llamar a la función
        text = componentes[componente]
    else:
        raise ValueError("Componente no válido. Debe ser 'Mano de obra', 'Materiales' o 'Nivel general'.")

    # Construyo la url de acuerdo al componente solicitado
    url = f"https://apis.datos.gob.ar/series/api/series?ids=Costo_construccion_{text}"

    # Manejo de errores: si la solicitud se procesó bien, importo la data. Sino, indico de qué error se trata
    try:
        response = requests.get(url)
        if response.status_code == 200:
            indices = response.json()  # Obtener los datos de la respuesta
            if 'data' in indices and len(indices['data'])>0:
                return indices['data']
            else:
                return "No hay datos disponibles"
        else:
            return "Error en la solicitud"
    except requests.RequestException:
        return "Error de conexión"


In [ ]:
# Llamo a la función y me guardo la respuesta (que viene por el return) en data
indices_mdo = get_costo_construccion("Mano de obra")
indices_mat = get_costo_construccion("Materiales")
indices_gral = get_costo_construccion("Nivel general")

In [ ]:
print(indices_mdo)
print(type(indices_mdo))

3. a) **Convertir los datos a un DataFrame de pandas** (5 puntos)
- Conviertan la respuesta de la API a un `DataFrame` de pandas.
- Verifiquen el formato de los datos y asegúrense de que están listos para su análisis.
- Incluyan las columnas necesarias para identificar el origen de los datos (por ejemplo, país, fecha, nombre del activo, etc.).

3. b) **Realizar una transformación de los datos** (5 puntos)

Apliquen alguna transformación relevante, como:
- Preservar sólo datos de un período específico o de un lugar específico.
- Crear una columna calculada (por ejemplo, una tasa de cambio o promedio).
- Convertir unidades de medida o redondear valores.
- Eliminar registros con datos faltantes.

In [ ]:
df_mdo = pd.DataFrame(indices_mdo, columns=["fecha", "indice_mdo"])
df_mat = pd.DataFrame(indices_mat, columns=["fecha", "indice_mat"])
df_gral = pd.DataFrame(indices_gral, columns=["fecha", "indice_gral"])

In [ ]:
df_indices = df_mdo.merge(df_mat, on="fecha", how="inner").merge(df_gral, on="fecha", how="inner")

In [ ]:
df_indices.head(5)

In [ ]:
print(df_mdo.shape)
print(df_mat.shape)
print(df_gral.shape) 
print(df_indices.shape) 
# Veo que los 3 df originales tienen la misma cantidad de filas que el combinado, por lo que no perdí ninguna fecha

In [ ]:
for column in df_indices.columns:
    print(f"Columna: {column}, Tipo: {df_indices[column].dtype}")

# Veo que la columna de fecha está como string, por lo que la transformo a formato fecha
df_indices["fecha"] = pd.to_datetime(df_indices["fecha"])

In [ ]:
for column in df_indices.columns:
    print(f"Columna: {column}, Tipo: {df_indices[column].dtype}")

In [ ]:
# Los datos son todos de un mismo origen, pero si se quisiera unir los índices de otros países luego al mismo df para hacer comparaciones
# podría ser útil incluir una columna llamada "pais" que identifique a las observaciones con valor "argentina". Esto sería así
df_indices["pais"]="Argentina"
df_indices.head(5)

In [ ]:
df_indices["var_mensual_mdo"] = (((df_indices["indice_mdo"]/df_indices["indice_mdo"].shift(1))-1)*100).round(2)
df_indices["var_mensual_mat"] = (((df_indices["indice_mat"]/df_indices["indice_mat"].shift(1))-1)*100).round(2)
df_indices["var_mensual_gral"] = (((df_indices["indice_gral"]/df_indices["indice_gral"].shift(1))-1)*100).round(2)

#df_indices["var_mensual_n_mdo"] = (df_indices["indice_mdo"]-df_indices["indice_mdo"].shift(1))
#df_indices["var_mensual_n_mat"] = (df_indices["indice_mat"]-df_indices["indice_mat"].shift(1))
#df_indices["var_mensual_n_gral"] = (df_indices["indice_gral"]-df_indices["indice_gral"].shift(1))

df_indices["incidencia_mdo"] = (df_indices["var_mensual_mdo"]*df_indices["indice_mdo"].shift(1))/(df_indices["indice_gral"].shift(1))
df_indices["incidencia_mat"] = (df_indices["var_mensual_mat"]*df_indices["indice_mat"].shift(1))/(df_indices["indice_gral"].shift(1))

# incidencia = var / t-1 * total (t-1)

In [ ]:
df_indices.head(5)

4. **Generar una Visualización de los Datos** (5 puntos)
    - Usen `matplotlib` o `seaborn` para crear un gráfico relevante, como:
        - Una serie de tiempo (si tienen datos temporales),
        - Un gráfico de barras para comparar categorías,
        - Un diagrama de dispersión o histograma, dependiendo de los datos,
        - Un boxplot,
        - Etc.
    - Asegúrense de etiquetar e incluir un título descriptivo en el gráfico.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
import locale

# Configurar el idioma en español
locale.setlocale(locale.LC_TIME, 'Spanish_Spain.1252')  # Para sistemas Linux y Mac
# Si estás en Windows, prueba con 'Spanish_Spain.1252'

In [ ]:
# Para graficar la serie de tiempo sería más útil tener una variable con el mes y el año, eliminando el día pues la medición corresponde
# siempre al mismo día del mes
x = df_indices['fecha'].dt.strftime('%b %Y')

y1 = df_indices["indice_mdo"]
y2 = df_indices["indice_mat"]
y3 = df_indices["indice_gral"]

In [ ]:
plt.plot(x, y1, color='C4')
plt.plot(x, y2, color='green')
plt.plot(x, y3, color='DarkBlue')

plt.xlabel('Período', color='0.2')
plt.ylabel('Índice', color='0.2')


plt.title("Índice de Costo de la Construcción por gran componente", 
          fontdict={
              'size': 14,
              'color': 'black',
              'weight': 'bold'
})


plt.gca().spines['top'].set_visible(False) 
plt.gca().spines['right'].set_visible(False)

x_ticks = x[::6]
plt.xticks(x_ticks, rotation=45, horizontalalignment="right")

plt.show()

In [ ]:
# Para graficar la serie de tiempo sería más útil tener una variable con el mes y el año, eliminando el día pues la medición corresponde
# siempre al mismo día del mes
x = df_indices['fecha'].dt.strftime('%b %Y')

y4 = df_indices["var_mensual_mdo"]
y5 = df_indices["var_mensual_mat"]
y6 = df_indices["var_mensual_gral"]

In [ ]:
plt.bar(x, y4, color='C4', label="Mano de obra")
plt.bar(x, y5, color='green', label="Materiales")
#plt.bar(x, y6, color='DarkBlue', label="Nivel general")

plt.xlabel('Período', color='0.2')
plt.ylabel('Índice', color='0.2')


plt.title("Índice de Costo de la Construcción por gran componente (Var. mensual)", 
          fontdict={
              'size': 14,
              'color': 'black',
              'weight': 'bold'
})


plt.gca().spines['top'].set_visible(False) 
plt.gca().spines['right'].set_visible(False)

x_ticks = x[::6]
plt.xticks(x_ticks, rotation=45, horizontalalignment="right")

plt.legend(loc='upper right')

plt.show()

In [ ]:
import seaborn as sns

In [ ]:
sns.set_theme(style="whitegrid", palette="pastel")

In [ ]:
df_indices_long = pd.melt(df_indices, id_vars="fecha", value_vars=["indice_mdo", "indice_mat", "indice_gral"])

In [ ]:
df_indices_long

In [ ]:
g = sns.lineplot(
        data=df_indices_long, 
        x="fecha", y="value", 
        hue = "variable"
    );
g = (g.set_axis_labels("Período", "Índice"))
plt.title("Índice de Costo de la Construcción por gran componente (Var. mensual)", 
          fontdict={
              'size': 14,
              'color': 'black',
              'weight': 'bold'
})
plt.show(g)

In [ ]:
df_vars_long = pd.melt(df_indices, id_vars="fecha", value_vars=["var_mensual_mdo", "var_mensual_mat"])

In [ ]:
df_vars_long

In [ ]:
sns.lineplot(
    data=df_vars_long, 
    x="fecha", y="value", 
    hue = "variable"
);
plt.show()#hue color de los puntos

___

#### Parte 2:
Aquí volvemos a las consignas definidas: vamos a utilizar un informe de ANSES (en formato PDF) así como información disponible en el sitio web de ANSES para resolver los siguientes problemas.

5. (10 puntos) Generar un código que les permita extraer la tabla de número personas con beneficios de desempleo de este informe de ANSES en formato PDF y generar un dataframe de Pandas. Para ello pueden usar la librería `tabula-py` que vimos en clase. Los valores numéricos deben quedar en un tipo de dato numérico.

In [ ]:
import tabula

In [ ]:
header = ["provincia",
         "beneficios", 
         "porcentaje"]
perimetro = (83.99*2.8346456693, 27.02*2.8346456693, 230.22*2.8346456693, 126.08*2.8346456693)
columnas = (68.35*2.8346456693, 94.90*2.8346456693, 126.08*2.8346456693)
lst_of_df = tabula.read_pdf("Informe-Estadisticas-SS-II-Trim-2024.pdf", output_format='dataframe', 
                            pages=21, area = perimetro, columns = columnas)

In [ ]:
lst_of_df

In [ ]:
tabla_p21 = lst_of_df[0]
tabla_p21.columns = header
tabla_p21

In [ ]:
for column in tabla_p21.columns:
    print(f"Columna: {column}, Tipo: {tabla_p21[column].dtype}")

In [ ]:
import re
regex = r'(\d+)%'
regex = re.compile (regex)

def extraer_porcentaje(valor):
    match = regex.search(valor)
    return match.group(1)  

# Uso extraer_porcentaje para reemplazar los valores de la columna "porcentaje" por un valor numérico
tabla_p21['porcentaje'] = tabla_p21['porcentaje'].apply(extraer_porcentaje)

In [ ]:
tabla_p21

In [ ]:
tabla_p21['beneficios'] = tabla_p21['beneficios'].astype(str)
tabla_p21['beneficios'] = tabla_p21['beneficios'].str.replace('.', '', regex=False)

# Convertimos a entero (opcional)
tabla_p21['beneficios'] = tabla_p21['beneficios'].astype(int)

In [ ]:
# Chequeo que la columna "porcentaje" haya quedado numérica
for column in tabla_p21.columns:
    print(f"Columna: {column}, Tipo: {tabla_p21[column].dtype}")

In [ ]:
tabla_p21

6. (10 puntos) Usando los datos anteriores, generar un mapa coloreado de las provincias de Argentina, donde las provincias con más desempleo estén en tonos de rojo y las provincias con menor desempleo estén en tonos de azul.

7. (20 puntos) Ahora utilicemos los datos de la siguiente página de ANSES: https://www.anses.gob.ar/oficinas-atencion-al-publico, donde detalla las ubicaciones de las oficinas de atención al público de ANSES. Los datos se muestran en una tabla, la cual está paginada de 1 a 8.

Nos encontramos con los siguientes desafíos: si prueban, van a notar que el sitio de ANSES no permite realizar consultas programáticas usando la librería `requests`, como veníamos haciendo. Por esa razón, vamos a tener que cambiar el enfoque: vamos a guardar el archivo HTML de la página localmente, y utilizar BeautifulSoup directamente en el contenido de este archivo (en lugar de hacer un `requests.get` a internet). El otro desafío es que el código HTML cambia con cada página. Deberán guardar un archivo HTML por cada una de las 8 páginas.

Finalmente, iteren sobre estos 8 archivos, extrayendo una a una las tablas. Finalmente, concatenaremos los resultados para generar un único dataframe con la información de todas las oficinas en cada fila.

8. (20 puntos) Ahora obtengamos información georreferenciada de las oficinas de ANSES. Para ello hay dos caminos:

- a) podemos usar la API de domicilios vista en la clase 6, pasarle las direcciones de la tabla generada en el punto anterior, y obtener la latitud y longitud de tantas oficinas como puedan.
- b) Alternativamente, notar que el texto "Ver el mapa" de la página apunta a un link de Google Maps que tiene incluidas la latitud y longitud (en la mayoria de los casos, en unos pocos el link no contiene esta información y a esos pueden descartarlos); por lo tanto, pueden también parsear el código HTML de las páginas (por ejemplo, con expresiones regulares, aunque no es la única forma) para así extraer estos valores. Finalmente, teniendo la latitud y longitud, crear una columna de `geometry` como hicimos en el caso de las propiedades del TP5, y con esto generar un Geodataframe.

_Ayuda para el camino b): pueden leer el contenido del HTML como texto plano con `.read()` y usar el método de strings `.split`. Notar que la parte de los links contiene una tag `href` que pueden usar para partir el string completo._

9. (15 puntos) Finalmente, generar un widget que nos permita elegir la provincia, y nos muestre un mapa de la provincia con las ubicaciones de las oficinas de ANSES del punto anterior (el resultado de superponer dos o más datasets georreferenciados en un mapa es lo que se llama un _overlay_).